### Заполнени пропусков

Часто в данных, с которыми необходимо работать, присутствуют пропуски, в результате чего аналитик оказывается перед выбором: игнорировать, отбросить или же заполнить пропущенные значения.

Не многие алгоритмы умеют работать с пропущенными значениями "из коробки", а реальный мир часто поставляет данные с пропусками. Одни из ключевых для анализа данных python библиотеки предоставляют простые как валенок решения: **pandas.DataFrame.fillna** и **sklearn.preprocessing.Imputer**.

Готовые библиотечные решения не прячут никакой магии за фасадом. Подходы к обработке отсутствующих значений напрашиваются на уровне здравого смысла («ad-hoc методы»):
- закодировать отдельным пустым значением типа "n/a" (для категориальных переменных);
- использовать наиболее вероятное значение признака (среднее или медиану для вещественных переменных, самое частое для категориальных);
- наоборот, закодировать каким-то невероятным значением (хорошо заходит для моделей, основанных на деревьях решений, т.к. позволяет сделать разделение на пропущенные и непропущенные значения);
- для упорядоченных данных (например, временных рядов) можно брать соседнее значение – следующее/предыдущее. (LOCF, Last observation carried forward)

Удобство использования библиотечных решений иногда подсказывает воткнуть что-то вроде **df = df.fillna(0)** и не париться о пропусках. Но это не самое разумное решение: большая часть времени обычно уходит не на построение модели, а на подготовку данных; бездумное неявное заполнение пропусков может спрятать баг в обработке и испортить модель.

Подроная [статья](https://basegroup.ru/community/articles/missing) как и когда стоит удалять/трансформировать пропуски

<center>Восстановление пропусков на основе регрессионных моделей</center>
<img src="img/regr_base.png">
Регрессионные методы представлены в  **sklearn** которые помогут востановить данные:
- SGD Regressor
- Lasso
- ElasticNet
- SVR kernel='rbf'
- Ensemble Regressors
- Ridge Regression
- SVR kernel='linear'

### Извлечение признаков (Feature Extraction)

Причины извлекать признаки:
- убрать шум, который может принять модел за сигнал и тем самым переобучаясь
- снизить вычислительную сложноость

Статистические подходы:
- Самый очевидный кандидат на отстрел – признак, у которого значение неизменно, т.е. не содержит вообще никакой информации. Если немного отойти от этого вырожденного случая, резонно предположить, что низковариативные признаки скорее хуже, чем высоковариативные. Так можно придти к идее отсекать признаки, дисперсия которых ниже определенной границы.


In [55]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import make_classification

x_data_generated, y_data_generated = make_classification()

In [51]:
x_data_generated.shape

(100, 20)

In [52]:
VarianceThreshold(.7).fit_transform(x_data_generated).shape

(100, 19)

In [53]:
VarianceThreshold(.8).fit_transform(x_data_generated).shape

(100, 19)

In [54]:
VarianceThreshold(.9).fit_transform(x_data_generated).shape

(100, 12)

Есть и другие способы, также основанные на классической статистике.

In [57]:
from sklearn.feature_selection import SelectKBest, f_classif

x_data_kbest = SelectKBest(f_classif, k=5).fit_transform(x_data_generated, y_data_generated)
x_data_varth = VarianceThreshold(.9).fit_transform(x_data_generated)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [58]:
cross_val_score(LogisticRegression(), x_data_generated, y_data_generated, scoring='neg_log_loss').mean()

-0.3460800117213228

In [59]:
cross_val_score(LogisticRegression(), x_data_kbest, y_data_generated, scoring='neg_log_loss').mean()

-0.27575116121167614

In [60]:
cross_val_score(LogisticRegression(), x_data_varth, y_data_generated, scoring='neg_log_loss').mean()

-0.27388845238799214

- Отбор с использованием моделей. Здесь другой подход: использовать какую-то baseline модель для оценки признаков, при этом модель должна явно показывать важность использованных признаков. Обычно используются два типа моделей: какая-нибудь "деревянная" композиция (например, Random Forest) или линейная модель с Lasso регуляризацией, склонной обнулять веса слабых признаков. Логика интутивно понятна: если признаки явно бесполезны в простой модели, то не надо тянуть их и в более сложную.

In [9]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

x_data_generated, y_data_generated = make_classification()

pipe = make_pipeline(SelectFromModel(estimator=RandomForestClassifier()),
                     LogisticRegression())

lr = LogisticRegression()
rf = RandomForestClassifier()

print(cross_val_score(lr, x_data_generated, y_data_generated, scoring='neg_log_loss').mean())
print(cross_val_score(rf, x_data_generated, y_data_generated, scoring='neg_log_loss').mean())
print(cross_val_score(pipe, x_data_generated, y_data_generated, scoring='neg_log_loss').mean())

-0.339702890809
-0.300563839288
-0.212944368093


In [10]:
x_data, y_data = get_data()
x_data = x_data.values

pipe1 = make_pipeline(StandardScaler(),
                      SelectFromModel(estimator=RandomForestClassifier()),
                      LogisticRegression())

pipe2 = make_pipeline(StandardScaler(),
                      LogisticRegression())

rf = RandomForestClassifier()

print('LR + selection: ', cross_val_score(pipe1, x_data, y_data, scoring='neg_log_loss').mean())
print('LR: ', cross_val_score(pipe2, x_data, y_data, scoring='neg_log_loss').mean())
print('RF: ', cross_val_score(rf, x_data, y_data, scoring='neg_log_loss').mean())


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


LR + selection:  -0.711260257044


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


LR:  -0.669572729709
RF:  -2.13626428692


- Перебор. Наконец, самый надежный, но и самый вычислительно сложный способ основан на банальном переборе: обучаем модель на подмножестве "фичей", запоминаем результат, повторяем для разных подмножеств, сравниваем качество моделей. Такой подход называется [Exhaustive Feature Selection](http://rasbt.github.io/mlxtend/user_guide/feature_selection/ExhaustiveFeatureSelector/).

Алгоритм перебора называется [SequentialFeatureSelector](http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/). Этот алгоритм можно развернуть: начинать с полного пространства признаков и выкидывать признаки по одному, пока это не портит качество модели или пока не достигнуто желаемое число признаков.

In [12]:
from mlxtend.feature_selection import SequentialFeatureSelector
selector = SequentialFeatureSelector(LogisticRegression(), scoring='neg_log_loss', verbose=2, k_features=3, forward=False, n_jobs=-1)
selector.fit(x_data_scaled, y_data)
selector.fit(x_data_scaled, y_data)

ModuleNotFoundError: No module named 'mlxtend'

Еще есть бибилотека для выбора фич
https://github.com/scikit-learn-contrib/boruta_py

### Нормализация и изменение распределения

Монотонное преобразование признаков критично для одних алгоритмов и не оказывает влияния на другие. Кстати, это одна из причин популярности деревьев решений и всех производных алгоритмов (случайный лес, градиентный бустинг) эти алгоритмы устойчивы к необычным распределениям.
Чаще всего необходимо адаптировать датасет под требования алгоритма. Параметрические методы обычно требуют как минимум симметричного и унимодального распределения данных, что не всегда обеспечивается реальным миром. Могут быть и более строгие требования (уместно вспомнить урок про линейные модели).
Впрочем, требования к данным предъявляют не только параметрические методы: тот же метод ближайших соседей предскажет полную чушь, если признаки ненормированы: одно распределение расположено в районе нуля и не выходит за пределы (-1, 1), а другой признак – это сотни и тысячи.

Самая простая трансформация – это Standart Scaling (она же Z-score normalization).

$$z = \frac{x - u}{\sigma}$$

StandartScaling хоть и не делает распределение нормальным в строгом смысле слова, но в какой-то мере защищает от выбросов

In [31]:
from sklearn.preprocessing import StandardScaler  
from scipy.stats import beta
from scipy.stats import shapiro

data = beta(1, 10).rvs(1000).reshape(-1, 1)

In [34]:
shapiro(data)
# значение статистики, p-value 

(0.8632318377494812, 1.2492330324561492e-28)

In [35]:
shapiro(StandardScaler().fit_transform(data))
# с таким p-value придется отклонять нулевую гипотезу о нормальности данных

(0.8632321357727051, 1.2493125974369767e-28)

In [42]:
data = np.array([1, 1, 0, -1, 2, 1, 2, 3, -2, 4, 100]).reshape(-1, 1).astype(np.float64)
StandardScaler().fit_transform(data)
(data - data.mean()) / data.std()

array([[-0.31922662],
       [-0.31922662],
       [-0.35434155],
       [-0.38945648],
       [-0.28411169],
       [-0.31922662],
       [-0.28411169],
       [-0.24899676],
       [-0.42457141],
       [-0.21388184],
       [ 3.15715128]])

Другой достаточно популярный вариант – MinMax Scaling, который переносит все точки на заданный отрезок (обычно (0, 1)).
$$X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}$$

In [45]:
from sklearn.preprocessing import MinMaxScaler
MinMaxScaler().fit_transform(data)
(data - data.min()) / (data.max() - data.min())

array([[ 0.02941176],
       [ 0.02941176],
       [ 0.01960784],
       [ 0.00980392],
       [ 0.03921569],
       [ 0.02941176],
       [ 0.03921569],
       [ 0.04901961],
       [ 0.        ],
       [ 0.05882353],
       [ 1.        ]])

StandartScaling и MinMax Scaling имеют похожие области применимости и часто сколько-нибудь взаимозаменимы. Впрочем, если алгоритм предполагает вычисление расстояний между точками или векторами, выбор по умолчанию – StandartScaling. Зато MinMax Scaling полезен для визуализации

Если мы предполагаем, что некоторые данные не распределены нормально, зато описываются логнормальным распределением, их можно легко привести к честному нормальному распределению:

In [47]:
from scipy.stats import lognorm
data = lognorm(s=1).rvs(1000)
shapiro(data)

(0.5775587558746338, 8.267660939516421e-44)

In [48]:
shapiro(np.log(data))

(0.9990927577018738, 0.9165956377983093)

Логнормальное распределение подходит для описания зарплат, стоимости ценных бумаг, населения городов, количества комментариев к статьям в интернете и т.п. Впрочем, для применения такого приема распределение не обязательно должно быть именно логнормальным – все распределения с тяжелым правым хвостом можно пробовать подвергнуть такому преобразованию.

In [1]:
from demo import get_data

x_data, y_data = get_data()
x_data.head(5)

,bathrooms,bedrooms,price,dishwasher,doorman,pets,air_conditioning,parking,balcony,bike,...,stainless,simplex,public,num_photos,num_features,listing_age,room_dif,room_sum,price_per_room,bedrooms_share
10,1.5,3,8.006368,0,0,0,0,0,0,0,...,0,0,0,5,0,515,1.5,4.5,666.666667,0.666667
10000,1.0,2,8.606119,0,1,1,0,0,0,0,...,0,0,0,11,57,527,1.0,3.0,1821.666667,0.666667
100004,1.0,1,7.955074,1,0,1,0,0,0,0,...,0,0,0,8,72,583,0.0,2.0,1425.000000,0.500000
100007,1.0,1,8.094073,0,0,0,0,0,0,0,...,0,0,0,3,22,582,0.0,2.0,1637.500000,0.500000
100013,1.0,4,8.116716,0,0,0,0,0,0,0,...,0,0,0,3,7,572,3.0,5.0,670.000000,0.800000


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
cross_val_score(LogisticRegression(), x_data, y_data, scoring='neg_log_loss').mean()
# кажется, что-то пошло не так! вообще-то стоит разобраться, в чем проблема

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


-0.68443637585039241

In [6]:
from sklearn.preprocessing import StandardScaler
cross_val_score(LogisticRegression(), StandardScaler().fit_transform(x_data), y_data, scoring='neg_log_loss').mean()
# ого! действительно помогает!

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


-0.66985157008143792

In [7]:
from sklearn.preprocessing import MinMaxScaler
cross_val_score(LogisticRegression(), MinMaxScaler().fit_transform(x_data), y_data, scoring='neg_log_loss').mean()
# a на этот раз – нет :( 

-0.68522641702184384

### Работа с сырыми данными